**DETECCIÓN DE LUNARES CANCERIGENOS**

*Presentado por: Juan Felipe Churio, Julian Motivar y Paula Ocampo*

Primero se tendrá que importar todas las librerias necesarias

In [3]:
import numpy as np
import pandas as pd
import os
import cv2
from skimage import feature
from sklearn import svm 
from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score
from skimage.feature import hog
from joblib import dump, load
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score




La clase "Image" es necesaria para tener un facil seguimiento del label y la imagen


In [4]:
class Image:
    def __init__(self, image, malicious):
        self.image = image
        self.malicious = malicious

folder_path = "/kaggle/input/umng-rp/images/train"
folder_path_test = "/kaggle/input/umng-rp/images/test"

En la siguiente se cargan todas las imagenes deseadas con la clase "Image" 

In [5]:
def processing():
    images = []
    benign_path = os.path.join(folder_path, "benign")
    malicious_path = os.path.join(folder_path, "malignant")

    for filename in os.listdir(benign_path):
        file_path = os.path.join(benign_path, filename)
        aux_image = cv2.imread(file_path, 0)
        new_image = Image(aux_image, 0)
        images.append(new_image)

    for filename in os.listdir(malicious_path):
        file_path = os.path.join(malicious_path, filename)
        aux_image = cv2.imread(file_path, 0)
        new_image = Image(aux_image, 1)
        images.append(new_image)

    return images
images=processing()

La siguiente funcion recolecta un vector de caracteristicas por cada Imagen, formando una matriz de caracteristicas junto con el arreglo de labels

In [6]:
def collecting_features(hog, images,cell_size,block_size,num_bins):
    hog_features = []
    lbp_hist = []
    labels = []
    features = []
   
    for image in images:
        feature_vector = feature.hog(image.image, orientations=num_bins, pixels_per_cell=cell_size, cells_per_block=block_size, visualize=False, block_norm='L2-Hys')
        
        lbp = feature.local_binary_pattern(image.image, 8, 1, method='uniform')
        hist, _ = np.histogram(lbp.ravel(), bins=int(lbp.max() + 1))
        concatenated_features = np.concatenate((feature_vector.ravel(), hist))
        features.append(concatenated_features)
        labels.append(image.malicious)

    features = np.array(features)
    labels = np.array(labels)
    
    return features,labels

En esta función se hace la validación cruzada, primero se divide el conjunto de imágenes en cuatro partes utilizando la función np.array_split(images, 4). Luego, se le asigna cada una de las partes a las variables parte_1, parte_2, parte_3 y parte_4.

Luego en el condicional se determina qué parte de las imágenes se utiliza como conjunto de prueba y cuáles se utilizan como conjunto de entrenamiento. Dependiendo del valor de test_number que se asigne a la función, se concatenan las partes correspondientes para formar el conjunto de entrenamiento llamado nuevo_arreglo y se llaman a la función collecting_features para asi  poder extraer las características HOG  y LBP y las etiquetas tanto para el conjunto de entrenamiento como para el conjunto de prueba. 

La función collecting_features se llama dos veces en cada condicional, una para el conjunto de entrenamiento y otra para el conjunto de prueba, utilizando diferentes parámetros de configuración para determinar cuales son los mas aptos para el testeo.

In [51]:
labals_predict=[]
for image  in images:
        labels_predict.append(image.malicious)
        
labels_1 = np.array(labels_predict[:660])
labels_2 = np.array(labels_predict[660:1319])
labels_3 = np.array(labels_predict[1319:1978])
labels_4 = np.array(labels_predict[1978:])

In [49]:
def Across_Validation(test_number,images):
    
    Split_Array= np.array_split(images.copy(),4)
    parte_1 = Split_Array[0]
    parte_2 = Split_Array[1]
    parte_3 = Split_Array[2]
    parte_4 = Split_Array[3]
    print(len(parte_1))
    print(len(parte_2))
    print(len(parte_3))
    print(len(parte_4))
    if test_number == 1:
        nuevo_arreglo = np.concatenate([parte_2, parte_3, parte_4])
        features_Across, labels_Across = collecting_features(hog, nuevo_arreglo,(8,8),(2,2),9)
        features_test_Across, labels_test_Across = collecting_features(hog, parte_1,(8,8),(2,2),9)
        
            
    if test_number == 2:
        nuevo_arreglo = np.concatenate([parte_1, parte_3, parte_4])
        features_Across, labels_Across = collecting_features(hog, nuevo_arreglo,(6, 16), (3, 3), 12)
        features_test_Across, labels_test_Across = collecting_features(hog, parte_2,(6, 16), (3, 3), 12)
        
    if test_number == 3:
        nuevo_arreglo = np.concatenate([parte_1, parte_2, parte_4])
        features_Across, labels_Across = collecting_features(hog, nuevo_arreglo,(8,8),(2,2),9)
        features_test_Across, labels_test_Across = collecting_features(hog, parte_3,(8,8),(2,2),9)
        
    if test_number == 4:
        nuevo_arreglo = np.concatenate([parte_1, parte_2, parte_3])
        features_Across, labels_Across = collecting_features(hog, nuevo_arreglo,(8,8),(2,2),9)
        features_test_Across, labels_test_Across = collecting_features(hog, parte_4,(8,8),(2,2),9)
        
    print(len(labels_predict))
    return features_Across, labels_Across, features_test_Across


features_Across = [0,0,0,0]
labels_Across= [0,0,0,0]
feature_test= [0,0,0,0]
predict=[0,0,0,0]
labels_predict = [0,0,0,0]
features_Across[0], labels_Across[0],feature_test[0] = Across_Validation(1,images)
features_Across[1], labels_Across[1],feature_test[1] = Across_Validation(2,images)
features_Across[2], labels_Across[2],feature_test[2] = Across_Validation(3,images)
features_Across[3], labels_Across[3],feature_test[3] = Across_Validation(4,images)

660
659
659
659
4
660
659
659
659
4
660
659
659
659
4
660
659
659
659
4


A continuación entrenamos y evalúamos el clasificador SVM con un kernel  de tipo lineal usando las características y etiquetas que obtuvimos en la validación cruzada.

Inicializamos las listas que se usan para almacenar los valores de las características, etiquetas, clasificadores SVM, etiquetas para cada prueba y los colores para las graficas de las curvas ROC.

Una vez el clasificador fue entrenado, relaizamos una predicción en el conjunto de prueba utilizando svm_classifier[i].predict(feature_test[i]) y liego lo almacenamos en la lista predict[i] para imprimir el resultado de la predicción.

Asi mismo se calcula el area bajo la curva AUC empleando las funciones roc_curve y roc_auc_score de la libreria scikit-learn, para poder graficar las curvas ROC y mostrar su respectiva AUC. 

Finalmente, se guardan los clasificadores previamente entrenados para cada caso de la validación.

In [59]:
features_np=[0,0,0,0]
labels_np=[0,0,0,0]
svm_classifier=[0,0,0,0]
ABCD=["A","B","C","D"]
colors=['#2E5063','#347E92','#2E5063','#347E92']
accurracy = [0,0,0,0]
predict = [0,0,0,0]
for i in range(4):
    features_np[i] = np.array(features_Across[i], dtype=np.float32)
    labels_np[i] = np.array(labels_Across[i])
    svm_classifier[i] = svm.SVC(kernel="linear")
    svm_classifier[i].fit(features_np[i], labels_np[i])

    predict[i] = svm_classifier[i].predict(feature_test[i])
    

    
    
    

In [73]:

print(len(predict[3]),len(labels_4))
labels_2 = labels_2[:len(predict[1])]
labels_4 = labels_4[:len(predict[3])]

acurracy1 = accuracy_score(labels_1, predict[0])
acurracy2 = accuracy_score(labels_2, predict[1])
acurracy3 = accuracy_score(labels_3, predict[2])
acurracy4 = accuracy_score(labels_4, predict[3])



print(acurracy1)
print(acurracy2)
print(acurracy3)
print(acurracy4)
    

659 659
0.7303030303030303
0.6843702579666161
0.6949924127465857
0.5569044006069803


**Resultados:**

Se puede ver que la implementación de la validación cruzada permitió evaluar la estabilidad y la calidad del clasificador al entrenarlo y evaluarlo en diferentes subconjuntos de datos y con diferentes valores de los hiperparámetros. En este caso el rendimiento del modelo es más consistente en todas las iteraciones del **svm_rpA** con un AUC de 0.9484619668830194 indica que el clasificador tiene un rendimiento muy bueno, ya que se acerca a 1.0. Esto significa que el clasificador tiene una alta capacidad para distinguir entre las clases positiva y negativa, por ende esto indica que este modelo es el más robusto. Asi mismo se puede observar que el modelo mas debil fue el **svm_rpB** con un AUC de 0.7988633555720286 que aunque también indica que el calsificador tuvo un buen rendimiento ,pero es ligeramente inferior al otro.

Por otro lado, se puede notar que al ajustar los parámetros del clasificador (Optimización de hiperparámetros), como el tipo de kernel, ya sea linear o radial, se encontró que la configuración óptima que maximiza el rendimiento del modelo es con un kernel lineal y parametros cell_size:(8,8),block_size:(2,2),num_bins:9, al contrario la peor combinación de la configuración fue con el kernel RBF y parametros cell_size:(6,16),block_size:(3,3),num_bins:12.

Finalmente, se puede decir que al combinar los descriptores LBP y HOG para la extracción de las características con el clasificador SVM, se pueden encontrar las características que son más importantes para la detección de lunares cancerígenos, De esta manera se busca comprender qué características anatomopatológicas de los lunares están correlacionadas con la malignidad. Sin embargo, es importante destacar que los resultados pueden variar según la calidad de los datos, la cantidad de datos de entrenamiento disponibles y las configuraciones de los hiperparámetros.


